# Week 3

In [69]:
import pandas as pd
import geocoder
import folium

## 1. Setting Variables

In [48]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
geopostal_data_filename = 'Geospatial_Coordinates.csv'

## 2. Using pd.read_html with header = 0.
This returns a list

In [74]:
html_list = pd.read_html(url, header = 0)
print(f'Type: {type(html_list)}, \t Lenght: {len(html_list)}')

Type: <class 'list'>, 	 Lenght: 3


## 3. Data is located at index 0. Setting data variable to the value located at index 0

In [73]:
data = html_list[0]
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 4. Removing all entries that contain "Not Assigned"

In [39]:
data_boroughs_cleaned = data[data.Borough != 'Not assigned']

In [41]:
data_boroughs_cleaned.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


## 5. Renaming all neighborhoods with name "Not Assigned" to the borough name.

In [56]:
data_neighborhood_renamed = data_boroughs_cleaned.copy()
data_neighborhood_renamed.Neighbourhood[data_neighborhood_renamed.Neighbourhood == 'Not assigned'] = data_neighborhood_renamed.Borough[data_neighborhood_renamed.Neighbourhood == 'Not assigned']

In [57]:
data_neighborhood_renamed.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


## 6. Loading Geopostal Data

In [58]:
geopostal_data = pd.read_csv(geopostal_data_filename, index_col='Postal Code')
geopostal_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


## 7. Using pandas.join to combine the geocoded data

In [67]:
geocoded_boroughs = data_neighborhood_renamed.join(geopostal_data, on='Postcode')
geocoded_boroughs.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Heights,43.718518,-79.464763
6,M6A,North York,Lawrence Manor,43.718518,-79.464763
7,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
9,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
10,M1B,Scarborough,Rouge,43.806686,-79.194353
11,M1B,Scarborough,Malvern,43.806686,-79.194353
13,M3B,North York,Don Mills North,43.745906,-79.352188


## 8. Plotting Map

In [84]:
toronot_map = folium.Map(location=[geocoded_boroughs.iloc[0].Latitude, geocoded_boroughs.iloc[0].Longitude], zoom_start=11)

# add markers to map
for index, row in geocoded_boroughs.iterrows():
    label = '{}, {}'.format(row.Neighbourhood, row.Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row.Latitude, row.Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronot_map)  
    
toronot_map

In [78]:
geocoded_boroughs.iterrows().__next__()

(2, Postcode                M3A
 Borough          North York
 Neighbourhood     Parkwoods
 Latitude            43.7533
 Longitude          -79.3297
 Name: 2, dtype: object)